Short Backwarding for Selecting the Best Historical Agent in a Consulting System for Portfolio Allocation with Deep Reinforcement Learning

## 6.0 Data Split
---

We will split both the close prices and the whole dataset into train and test (trade) data.

We will use 80% of the data for training and then test on the remaining 20%.

We will make use of the FinRL Library function of data_split to split our data into train and test

### 6.1 Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import ta
from ta import add_all_ta_features
from ta.utils import dropna
from finrl.preprocessing.data import data_split
from finrl.preprocessing.preprocessors import FeatureEngineer

In [2]:
df_close_full_stocks = pd.read_csv('datasets/close_prices.csv')

### 6.2 Load the data

In [3]:
filtered_stocks = pd.read_csv('filtered_stocks.csv')
# df_close_full_stocks = pd.read_csv('datasets/close_prices.csv')
data_df = pd.read_csv('data_df.csv')
filtered_stocks = filtered_stocks.drop(columns=['Unnamed: 0'])
filtered_stocks = filtered_stocks['stock_name'].tolist()
data_df = data_df.drop(columns=['Unnamed: 0'])

#%store filtered_stocks
#%store df_close_full_stocks
#%store data_df

%store -r filtered_stocks
# %store -r df_close_full_stocks
%store -r data_df

In [4]:
data_df.head()

,date,tic,close,high,low,open,volume,cov_list,f01,f02,f03,f04
0,2009-03-20,AXP,9.658469,13.190000,12.120000,13.190000,31088200.0,"[[0.0026107181410731633, 0.0012702991646503876...",0.218444,1.06041,5.250239,1.707034
1,2009-03-20,CAT,17.987209,28.900000,26.730000,28.629999,16531300.0,"[[0.0026107181410731633, 0.0012702991646503876...",0.218444,1.06041,5.250239,1.707034
2,2009-03-20,CSCO,10.789354,16.570000,15.750000,16.370001,66078200.0,"[[0.0026107181410731633, 0.0012702991646503876...",0.218444,1.06041,5.250239,1.707034
3,2009-03-20,CVX,35.435562,67.980003,64.269997,67.540001,23811700.0,"[[0.0026107181410731633, 0.0012702991646503876...",0.218444,1.06041,5.250239,1.707034
4,2009-03-20,DIS,14.977372,17.980000,17.080000,17.799999,17766600.0,"[[0.0026107181410731633, 0.0012702991646503876...",0.218444,1.06041,5.250239,1.707034


In [5]:
df_close_full_stocks.head()

,date,MCD,CVX,DD,PG,UNH,V,WBA,RTX,INTC,...,AAPL,GS,IBM,TRV,HD,CAT,XOM,DIS,KO,BA
0,2008-03-19,34.153214,43.431351,32.355705,42.133186,28.681099,12.605218,22.749361,29.209345,13.014488,...,3.915352,127.940323,64.913177,31.375137,17.912857,47.481430,46.908859,26.417377,18.233225,54.094543
1,2008-03-20,34.636936,44.131432,32.417919,43.102497,28.495552,14.356561,22.999493,29.315552,13.421770,...,4.024055,138.037750,65.684776,32.308086,18.727085,47.539391,47.225548,26.975491,18.561644,55.088791
2,2008-03-24,35.496170,44.555729,33.466522,43.475304,28.810194,13.325832,24.143843,29.880541,13.656266,...,4.213072,137.461456,66.089973,32.240482,19.527952,48.962620,47.753334,27.093884,18.604212,56.186138
3,2008-03-25,35.769848,44.836826,33.644249,43.208118,28.463276,14.111155,23.768646,29.804058,13.742652,...,4.256853,138.037750,65.484924,32.287811,19.194254,49.355450,47.336636,27.127707,18.674158,55.898918
4,2008-03-26,35.387978,45.059536,33.599819,43.270256,27.511274,14.269550,23.737381,29.570433,13.489651,...,4.380049,134.848679,64.896484,32.145817,18.793810,49.819118,47.925571,26.857107,18.595097,56.193501


In [6]:
# Close Prices data frame

# Reset the Index to tic and date
df_prices = data_df.reset_index().set_index(['tic', 'date']).sort_index()

# Get all the Close Prices
df_close = pd.DataFrame()

for ticker in filtered_stocks:
    series = df_prices.xs(ticker).close
    df_close[ticker] = series

In [7]:
df_close.head()

,JNJ,PG,WMT,PFE,KO,MMM,MCD,VZ,IBM,RTX,MRK,HD,CSCO,XOM,DIS,CVX,AXP,CAT,MSFT,NKE
date,,,,,,,,,,,,,,,,,,,,
2009-03-20,32.971889,29.053089,11.836151,7.213283,13.396739,21.840876,34.913979,12.897711,52.335407,17.653549,15.203940,15.388922,10.789354,37.472710,14.977372,35.435562,9.658469,17.987209,12.683448,9.356719
2009-03-23,33.967377,30.423218,12.287254,7.419683,13.868021,23.452911,36.200291,13.533669,55.842911,18.938465,15.920894,16.145868,11.514978,39.990177,16.239073,37.866932,11.470416,19.688248,13.627650,9.897091
2009-03-24,33.629173,29.996250,12.191783,7.366758,13.830312,22.967873,35.150246,13.289741,55.611000,18.820860,15.790021,15.937532,11.311531,39.338127,15.698346,37.768353,10.950462,19.535419,13.330266,9.477262
2009-03-25,33.731266,30.627134,12.334997,7.546691,14.059669,23.310259,36.101837,13.132926,55.412960,19.029940,15.647769,16.166708,11.236934,39.780388,15.904337,38.168110,11.115903,19.209829,13.293093,9.637298
2009-03-26,33.756805,31.073229,12.592767,7.610199,14.091089,24.266064,36.790947,13.302806,55.882511,19.805246,15.693288,16.763929,11.738763,40.387077,16.359232,38.425495,11.919463,20.452393,13.999379,9.892933


In [8]:
df_close = df_close.reset_index()

### 6.3 Split the Data

In [9]:
# Define the start and end dates for the train and test data

train_pct = 0.8 # percentage of train data

date_list = list(data_df.date.unique()) # List of dates in the data

date_list_len = len(date_list) # len of the date list
train_data_len = int(train_pct * date_list_len) # length of the train data

train_start_date = date_list[0]
train_end_date = date_list[train_data_len]

test_start_date = date_list[train_data_len+1]
test_end_date = date_list[-1]

In [10]:
print('Training Data: ', 'from ', train_start_date, ' to ', train_end_date)

Training Data:  from  2009-03-20  to  2021-03-26


In [11]:
print('Testing Data: ', 'from ', test_start_date, ' to ', test_end_date)

Testing Data:  from  2021-03-29  to  2024-03-28


In [12]:
df_close_full_stocks

,date,MCD,CVX,DD,PG,UNH,V,WBA,RTX,INTC,...,AAPL,GS,IBM,TRV,HD,CAT,XOM,DIS,KO,BA
0,2008-03-19,34.153214,43.431351,32.355705,42.133186,28.681099,12.605218,22.749361,29.209345,13.014488,...,3.915352,127.940323,64.913177,31.375137,17.912857,47.481430,46.908859,26.417377,18.233225,54.094543
1,2008-03-20,34.636936,44.131432,32.417919,43.102497,28.495552,14.356561,22.999493,29.315552,13.421770,...,4.024055,138.037750,65.684776,32.308086,18.727085,47.539391,47.225548,26.975491,18.561644,55.088791
2,2008-03-24,35.496170,44.555729,33.466522,43.475304,28.810194,13.325832,24.143843,29.880541,13.656266,...,4.213072,137.461456,66.089973,32.240482,19.527952,48.962620,47.753334,27.093884,18.604212,56.186138
3,2008-03-25,35.769848,44.836826,33.644249,43.208118,28.463276,14.111155,23.768646,29.804058,13.742652,...,4.256853,138.037750,65.484924,32.287811,19.194254,49.355450,47.336636,27.127707,18.674158,55.898918
4,2008-03-26,35.387978,45.059536,33.599819,43.270256,27.511274,14.269550,23.737381,29.570433,13.489651,...,4.380049,134.848679,64.896484,32.145817,18.793810,49.819118,47.925571,26.857107,18.595097,56.193501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4030,2024-03-22,282.630005,153.113876,75.739998,160.622849,490.070007,282.736755,20.296997,94.968666,42.397793,...,172.046646,406.820007,188.964172,224.449997,390.279999,356.809326,112.575607,115.870003,60.490002,188.850006
4031,2024-03-25,278.619995,154.905777,75.940002,159.162277,485.880005,280.690521,20.346308,95.058121,41.660786,...,170.618591,404.940002,186.934326,225.529999,383.510010,354.657196,113.726265,119.360001,60.400002,191.410004
4032,2024-03-26,278.619995,153.717773,75.540001,159.519974,492.309998,280.081665,20.227959,95.485550,41.820141,...,169.480133,406.179993,186.647186,225.330002,379.929993,355.095612,112.873192,119.930000,60.540001,187.500000
4033,2024-03-27,282.019989,154.786987,76.500000,161.566742,493.100006,278.504578,20.730946,96.867241,43.592937,...,173.075241,415.250000,188.924576,228.830002,385.890015,363.325592,114.043686,120.980003,61.029999,191.949997


In [13]:
# Split the whole dataset
train_data = data_split(data_df, train_start_date, train_end_date)
test_data = data_split(data_df, test_start_date, test_end_date)

# Split the Close Prices dataset
prices_train_data = df_close[df_close['date']<=train_end_date]
prices_test_data = df_close[df_close['date']>=test_start_date]

# split the Close Prices of all stocks
prices_full_train = df_close_full_stocks[df_close_full_stocks['date']<=train_end_date]
prices_full_test = df_close_full_stocks[df_close_full_stocks['date']>=test_start_date]

### 6.4 Store the Dataframes

In [14]:
prices_train = prices_train_data.copy()
prices_test = prices_test_data.copy()

train_df = train_data.copy()
test_df = test_data.copy()

prices_full_train_df = prices_full_train.copy()
prices_full_test_df = prices_full_test.copy()

In [15]:
prices_train.to_csv('./prices_train.csv')
prices_test.to_csv('./prices_test.csv')

train_df.to_csv('./train_df.csv')
test_df.to_csv('./test_df.csv')

prices_full_train_df.to_csv('./prices_full_train_df.csv')
prices_full_train_df.to_csv('./prices_full_test_df.csv')

%store prices_train
%store prices_test

%store train_df
%store test_df

%store prices_full_train_df
%store prices_full_test_df

Stored 'prices_train' (DataFrame)
Stored 'prices_test' (DataFrame)
Stored 'train_df' (DataFrame)
Stored 'test_df' (DataFrame)
Stored 'prices_full_train_df' (DataFrame)
Stored 'prices_full_test_df' (DataFrame)


In [16]:
%store df_close_full_stocks

Stored 'df_close_full_stocks' (DataFrame)
